In [2]:
 import implicit
import numpy as np
import time

In [3]:
from implicit.datasets.movielens import get_movielens

titles, ratings = get_movielens(variant="20m")

In [4]:
user_ratings = ratings.T.tocsr()

In [5]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50)

In [6]:
model.fit(user_ratings)

  0%|          | 0/15 [00:00<?, ?it/s]

In [15]:
user_ratings.toarray()[1][2]

3.5

In [22]:
arr = user_ratings.toarray()
u1 = arr[1]
rows = []
cols = []
data = []
print(u1)
for i in range(len(u1)):
    if u1[i] > 0:
        cols.append(i)
        data.append(1)
        rows.append(1)
print(cols)

[0.  0.  3.5 ... 0.  0.  0. ]
[2, 29, 32, 47, 50, 112, 151, 223, 253, 260, 293, 296, 318, 337, 367, 541, 589, 593, 653, 919, 924, 1009, 1036, 1079, 1080, 1089, 1090, 1097, 1136, 1193, 1196, 1198, 1200, 1201, 1208, 1214, 1215, 1217, 1219, 1222, 1240, 1243, 1246, 1249, 1258, 1259, 1261, 1262, 1266, 1278, 1291, 1304, 1321, 1333, 1348, 1350, 1358, 1370, 1374, 1387, 1525, 1584, 1750, 1848, 1920, 1967, 1994, 1997, 2021, 2100, 2118, 2138, 2140, 2143, 2173, 2174, 2193, 2194, 2253, 2288, 2291, 2542, 2628, 2644, 2648, 2664, 2683, 2692, 2716, 2761, 2762, 2804, 2872, 2918, 2944, 2947, 2959, 2968, 3000, 3030, 3037, 3081, 3153, 3265, 3438, 3476, 3479, 3489, 3499, 3889, 3932, 3996, 3997, 4011, 4027, 4105, 4128, 4133, 4226, 4306, 4446, 4467, 4571, 4720, 4754, 4878, 4896, 4911, 4915, 4941, 4980, 4993, 5026, 5039, 5040, 5146, 5171, 5540, 5679, 5797, 5816, 5898, 5952, 5999, 6093, 6242, 6333, 6502, 6539, 6754, 6755, 6774, 6807, 6834, 6888, 7001, 7045, 7046, 7153, 7164, 7247, 7387, 7389, 7438, 7449, 7454, 

In [23]:
from scipy.sparse import coo_matrix, csr_matrix
user_items = coo_matrix((data, (rows, cols)), shape=(len(arr), len(arr[0])))

In [29]:
predictions = []
for i in cols:
    predictions.append(model.explain(1, user_items, i, N = len(cols))[0])

In [30]:
predictions

[0.32612432957673265,
 0.39180820744662626,
 0.5335821816139533,
 0.6273117733987913,
 0.6360114274816121,
 0.08487080341621406,
 0.3481636243466467,
 0.29119766130703534,
 0.573005237579163,
 0.6780665358486693,
 0.6264194001157131,
 0.5036603921494106,
 0.42002207831634936,
 0.40620380902311726,
 0.23442472250155286,
 0.8235717409014434,
 0.4206752572070571,
 0.4992590242262832,
 0.3756804371923419,
 0.3465824555714051,
 0.4270441879149904,
 0.09445838740715388,
 0.6191262769598005,
 0.469504252087778,
 0.7471757210078817,
 0.6486697813826418,
 0.3908275257618012,
 0.3565023547493731,
 0.8578722873719474,
 0.46933142431811453,
 0.7541112935510895,
 0.8133099358999811,
 0.8074147812632766,
 0.5245907930447262,
 0.6135394203478719,
 0.8670297360221254,
 0.5042016763566814,
 0.17938278738209293,
 0.5611292076867349,
 0.6089976969195141,
 0.6642692355521221,
 0.17229900186062702,
 0.35575906494896414,
 0.27471040295377774,
 0.865730040912778,
 0.4723166692099769,
 0.5257333436691505,
 0.

In [31]:
rounded_predictions = [round(i) for i in predictions]

In [34]:
from sklearn.metrics import mean_squared_error
mean_squared_error(predictions, [1 for i in range(len(cols))])

0.49800675917114023

In [30]:
from scipy.sparse import coo_matrix

In [31]:
user_index = np.array([0])
movie_index = np.array([3])

st = time.time()
user_items = coo_matrix((np.array([1]), (user_index, movie_index)), shape=user_ratings[0].toarray().shape).tocsr()
print("100k")
print("matrix time", time.time() - st)
st = time.time()
model.partial_fit_users(user_index, user_items)
print("fit time", time.time() - st)

100k
matrix time 0.0017802715301513672
fit time 0.00908803939819336


In [32]:
user_index = np.array([0])
movie_index = np.array([3])

st = time.time()
user_items = coo_matrix((np.array([1]), (user_index, movie_index)), shape=user_ratings[0].toarray().shape).tocsr()
print("100k")
print("matrix time", time.time() - st)
st = time.time()
model.partial_fit_items(user_index, user_items)
print("fit time", time.time() - st)

100k
matrix time 0.001255035400390625
fit time 0.008704185485839844


In [38]:
ids, scores = model.recommend(0, user_ratings[0], N=10, filter_already_liked_items=False)

In [40]:
ids

array([   3,  289,  235,  358,  825,  108,  316,  763, 1010,  789],
      dtype=int32)

In [41]:
scores

array([0.04815494, 0.02354678, 0.02318918, 0.02315067, 0.02249356,
       0.02130273, 0.02120527, 0.02057432, 0.02054867, 0.01968906],
      dtype=float32)

In [26]:
user_ratings.shape

(138494, 131263)